In [4]:
import pandas as pd
import glob

files = glob.glob('output_bounds2_gurobi/*.txt')

gene = [x.split('/')[1].split('_')[1] for x in files]
coenzyme = [x.split('/')[1].split('_')[2] for x in files]
stoich = [x.split('/')[1].split('_')[3].split('.')[0] for x in files]
fdata = pd.DataFrame({'file': files, 'gene': gene, 'coenzyme': coenzyme, 'stoich': stoich})

fdata_s = fdata[fdata['stoich'] == 'S'].sort_values('gene')
fdata_st = fdata[fdata['stoich'] == 'ST'].sort_values('gene')

dfs = [];
for idx,row in fdata_s.iterrows():
    df = pd.read_csv(row.file,sep='\t',index_col=0)
    df = df.set_index(['carbon','nitrogen','electron'])[['growth_rate']]
    df.columns = [row.gene]
    dfs.append(df)
    
dfs = pd.concat(dfs,axis=1)

dfst = [];

for idx,row in fdata_st.iterrows():
    df = pd.read_csv(row.file,sep='\t',index_col=0)
    df = df.set_index(['carbon','nitrogen','electron'])[['growth_rate']]
    df.columns = [row.gene]
    dfst.append(df)

dfst = pd.concat(dfst,axis=1)

In [7]:

cutoff = 1e-4
wt = pd.read_csv('output_gurobi/iJO1366.txt',sep='\t')[['carbon','nitrogen','electron','growth_rate']]
wt.set_index(['carbon','nitrogen','electron'],inplace=True)
wt = wt[wt['growth_rate'] > cutoff]
dfst = dfst.loc[wt.index]
dfs = dfs.loc[wt.index]

dfst = dfst.join(wt)
dfs = dfs.join(wt)

normalized = []
genes = [x for x in list(dfst) if x != 'growth_rate']
for gene in genes:
    normalized.append(dfst[gene].values / dfst['growth_rate'].values)
    
dfst_normalized = pd.DataFrame(normalized,columns=dfst.index,index = genes).T


normalized = []
genes = [x for x in list(dfs) if x != 'growth_rate']
for gene in genes:
    normalized.append(dfs[gene].values / dfs['growth_rate'].values)
    
dfs_normalized = pd.DataFrame(normalized,columns=dfs.index,index = genes).T

In [13]:
(dfst_normalized < cutoff).sum()

b0031     0
b0116     0
b0356     0
b0414     0
b1479     0
b1761     0
b1852     0
b2040     0
b2052     0
b2705    12
b2867     0
b3213     0
b3553     0
b3600    12
b3608     0
b3844     0
dtype: int64

In [15]:
dfst_normalized.to_hdf('relativeGrowthRates.StoichAndThermo.MR2.hdf','df')



In [52]:
dfst_normalized[(dfst_normalized.index.get_level_values(2) == 'EX_o2_e') & (dfst_normalized.index.get_level_values(1) == 'EX_nh4_e')]['b3600'].sort_values().dropna()

carbon        nitrogen  electron
EX_asn__L_e   EX_nh4_e  EX_o2_e     1.000000
EX_acald_e    EX_nh4_e  EX_o2_e     1.000000
EX_glyc__R_e  EX_nh4_e  EX_o2_e     1.000000
EX_rmn_e      EX_nh4_e  EX_o2_e     1.000000
EX_hxan_e     EX_nh4_e  EX_o2_e     1.000000
                                      ...   
EX_xylu__L_e  EX_nh4_e  EX_o2_e     1.000000
EX_idon__L_e  EX_nh4_e  EX_o2_e     1.000000
EX_fald_e     EX_nh4_e  EX_o2_e     1.000000
EX_ala__L_e   EX_nh4_e  EX_o2_e     1.000000
EX_for_e      EX_nh4_e  EX_o2_e     1.000002
Name: b3600, Length: 179, dtype: float64

In [57]:
dfst_normalized.mean().sort_values()

b1761    0.930053
b3844    0.987087
b1852    0.988460
b3608    0.993339
b3600    0.999691
b2705    0.999691
b2052    1.000000
b3213    1.000000
b3553    1.000000
b1479    1.000000
b2040    1.000000
b0414    1.000001
b0356    1.000021
b0031    1.000705
b2867    1.004000
b0116    1.239500
dtype: float64

In [35]:
dfst_normalized.reset_index()[dfst_normalized.reset_index().nitrogen.isin(['EX_nh4_e'])].mean()

b0031    1.000396
b0116    1.225692
b0356    1.000516
b0414    1.000000
b1479    1.000000
b1761    0.937371
b1852    0.997823
b2040    1.000000
b2052    1.000000
b2705    0.999937
b2867    1.000544
b3213    1.000000
b3553    1.000000
b3600    0.999937
b3608    0.993694
b3844    0.997615
dtype: float64

In [36]:
dfst_normalized.mean()

b0031    1.000705
b0116    1.239500
b0356    1.000021
b0414    1.000001
b1479    1.000000
b1761    0.930053
b1852    0.988460
b2040    1.000000
b2052    1.000000
b2705    0.999691
b2867    1.004000
b3213    1.000000
b3553    1.000000
b3600    0.999691
b3608    0.993339
b3844    0.987087
dtype: float64

In [32]:
dfst

b0031      b0116      b0356  \
carbon        nitrogen    electron                                     
EX_12ppd__S_e EX_23camp_e EX_tmao_e  38.150316  43.264701  38.051294   
                          EX_dmso_e  37.434943  41.311576  37.347595   
                          EX_fum_e   15.841827  22.526548  15.799150   
                          EX_o2_e    54.323464  55.856093  54.323464   
                          EX_no2_e   50.436373  50.744040  50.402517   
...                                        ...        ...        ...   
EX_xylu__L_e  EX_xtsn_e   EX_fum_e   30.113751  37.387008  30.103441   
                          EX_o2_e    58.044068  58.901904  58.044068   
                          EX_no2_e   54.767088  55.164243  54.734496   
                          EX_no3_e   57.319011  57.739266  57.305369   
                          None       26.399593  35.084701  26.388625   

                                         b0414      b1479      b1761  \
carbon        nitrogen    electron                                     
EX_12ppd__S_e EX_23camp_e EX_tmao_e  38.051413  38.051294  34.523469   
                          EX_dmso_e  37.347699  37.347595  33.888613   
                          EX_fum_e   15.799201  15.799150  14.284192   
                          EX_o2_e    54.323464  54.323464  48.894851   
                          EX_no2_e   50.402558  50.402517  45.254026   
...                                        ...        ...        ...   
EX_xylu__L_e  EX_xtsn_e   EX_fum_e   30.103453  30.103441  27.065415   
                          EX_o2_e    58.044068  58.044068  52.638468   
                          EX_no2_e   54.734535  54.734496  50.415335   
                          EX_no3_e   57.305385  57.305369  52.445847   
                          None       26.388638  26.388625  23.404793   

                                         b1852      b2040      b2052  \
carbon        nitrogen    electron                                     
EX_12ppd__S_e EX_23camp_e EX_tmao_e  37.151179  38.051294  38.051294   
                          EX_dmso_e  36.554533  37.347595  37.347595   
                          EX_fum_e   15.799150  15.799150  15.799150   
                          EX_o2_e    54.323464  54.323464  54.323464   
                          EX_no2_e   49.156806  50.402517  50.402517   
...                                        ...        ...        ...   
EX_xylu__L_e  EX_xtsn_e   EX_fum_e   27.368174  30.103441  30.103441   
                          EX_o2_e    58.044068  58.044068  58.044068   
                          EX_no2_e   53.130693  54.734496  54.734496   
                          EX_no3_e   56.903435  57.305369  57.305369   
                          None       23.743617  26.388625  26.388625   

                                         b2705      b2867      b3213  \
carbon        nitrogen    electron                                     
EX_12ppd__S_e EX_23camp_e EX_tmao_e  38.051294  38.051294  38.051294   
                          EX_dmso_e  37.347595  37.347595  37.347595   
                          EX_fum_e   15.799150  15.799150  15.799150   
                          EX_o2_e    54.323464  54.323464  54.323464   
                          EX_no2_e   50.402517  50.402517  50.402517   
...                                        ...        ...        ...   
EX_xylu__L_e  EX_xtsn_e   EX_fum_e   30.103441  32.403172  30.103441   
                          EX_o2_e    58.044068  58.501540  58.044068   
                          EX_no2_e   54.734496  55.151401  54.734496   
                          EX_no3_e   57.305369  57.687200  57.305369   
                          None       26.388625  28.543981  26.388625   

                                         b3553      b3600      b3608  \
carbon        nitrogen    electron                                     
EX_12ppd__S_e EX_23camp_e EX_tmao_e  38.051294  38.051294  38.088346   
                          EX_dmso_e  37.347595  37.347595  37.380284   
     